# SENet通道注意力机制

建模出不同通道之间的相关性，通过网络学习的方式来自动获取到每个特征通道的重要程度，最后再为每个通道赋予不同的权重系数，从而来强化重要的特征抑制非重要的特征。

## 1. 计算过程

* 1. 全局平均池化

    将每个通道的二维特征(H*W)压缩为1个实数，将特征图从 (B, C, H, W) ----> (B, C, 1, 1)

* 2. 给每个特征通道生成一个权重值

    通过两个全连接层构建通道间的相关性，输出的权重值数目和输入特征图的通道数相同

* 3. 归一化权重加权到每个通道的特征上

    逐通道乘以权重系数

## 2. 代码实现

In [14]:
from torch import nn
import torch


class SEBlock(nn.Module):
    """通道注意力机制

    Args:
        nn (_type_): _description_
    """

    def __init__(self, channel, reduction=8, bias=False):
        super(SEBlock, self).__init__()
        # 先在H*W维度进行压缩，全局平均池化将每个通道平均为一个值
        # (B, C, H, W) ---- (B, C, 1, 1)
        self.avg_pool = nn.AdaptiveAvgPool2d(1)  # 自适应全局池化

        # 利用各channel维度的相关性计算权重
        # (B, C) --- (B, C//K) --- (B, C) --- sigmoid
        self.fc1 = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=bias),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=bias),
            nn.Sigmoid(),
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        print(y.shape)
        y_out = self.fc1(y).view(b, c, 1, 1)
        return x * y_out

### 3. 验证

In [15]:
se_block = SEBlock(64)

x = torch.randn(1, 64, 102, 102)

y = se_block.forward(x)

print(x.shape)
print(y.shape)

torch.Size([1, 64])
torch.Size([1, 64, 102, 102])
torch.Size([1, 64, 102, 102])
